# Start and Test Rekognition Model

## Upgrade awscli and boto3 librariies

In [ ]:
!pip3 install boto3 --upgrade --user

In [ ]:
!pip3 install awscli --upgrade --user

## 1- Start your Rekognition model (Editing required)

In [ ]:
!aws rekognition start-project-version \
  --project-version-arn "arn:aws:rekognition:us-east-1:745137163025:project/dec-20/version/dec-20.2019-12-20T09.52.19/1576857139822" \
  --min-inference-units 1 \
  --endpoint-url https://rekognition.us-east-1.amazonaws.com \
  --region us-east-1

## 2- Define inference function (No editing required)

In [ ]:
confidence =98

import boto3
import io
%matplotlib inline
from PIL import Image, ImageDraw, ExifTags, ImageColor, ImageFont

def show_custom_labels(model,bucket,photo):
     

    client=boto3.client('rekognition')

    # Load image from S3 bucket
    s3_connection = boto3.resource('s3')

    s3_object = s3_connection.Object(bucket,photo)
    s3_response = s3_object.get()

    stream = io.BytesIO(s3_response['Body'].read())
    image=Image.open(stream)
    
    #Call DetectCustomLabels 
    response = client.detect_custom_labels(Image={'S3Object': {'Bucket': bucket, 'Name': photo}},
        MinConfidence=1,
        ProjectVersionArn=model)
   
   
    imgWidth, imgHeight = image.size  
    draw = ImageDraw.Draw(image)  

    colors = ('#FF0000','#0000FF','#FF00FF')
    dict1 = {'ava':0,'azt':1,'eli':2}
    # calculate and display bounding boxes for each detected custom label       
    for customLabel in response['CustomLabels']:
        if customLabel['Confidence'] >= confidence:
            if 'Geometry' in customLabel:
                box = customLabel['Geometry']['BoundingBox']
                left = imgWidth * box['Left']
                top = imgHeight * box['Top']
                width = imgWidth * box['Width']
                height = imgHeight * box['Height']
                draw.text((left + width-20,top-15), customLabel['Name'], fill= colors[dict1[customLabel['Name']]])
                draw.text((left,top-15), str(customLabel['Confidence'])[:6], fill= colors[dict1[customLabel['Name']]])#, font='Arial.ttf') 



                points = (
                    (left,top),
                    (left + width, top),
                    (left + width, top + height),
                    (left , top + height),
                    (left, top))
                draw.line(points, fill= colors[dict1[customLabel['Name']]], width=5)


    image.show()
    display(image)
    image = image.convert('RGB')
    image.save('inference.jpg',optimize=True,quality=300)

    return len(response['CustomLabels'])

## 3- Detect medicines (Editing required)

In [ ]:
bucket="your bucket name where you stored your images"
model= "Your model arn. It should be similar to below exapmple"
photo = 'path to picture in your bucket'


# Example:

# bucket = 'bms-od'
# model = 'arn:aws:rekognition:us-east-1:745137163025:project/dec-20/version/dec-20.2019-12-20T09.52.19/1576857139822'
# photo = 'pictures/1.png'

label_count=show_custom_labels(model,bucket,photo)